# Set up

### Import Python packages to use in the script

In [ ]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
import pandas as pd
import pyclesperanto as cle
import skan
from imaris_ims_file_reader.ims import ims
from tqdm import tqdm

### Define the functions that will run different steps/tasks on the data

Need to select which GPU device to send data to

In [ ]:
cle.select_device("NVIDIA")

In [ ]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [ ]:
def get_measurements(mask,img,props):
    filtered_masks = sk.segmentation.clear_border(mask)
    size_filtered = sk.morphology.remove_small_objects(filtered_masks,min_size=2000)
    df = sk.measure.regionprops_table(size_filtered,img,properties=props)
    df = pd.DataFrame.from_dict(df)
    return size_filtered, df

In [ ]:
def scaled_vol(df,voxel):
    scaled_vol = []
    for a in np.asarray(df['area']).astype(int):
        scaled_vol.append(a*voxel)
    vol_df = pd.Series(scaled_vol,name='Volume (um^3)')
    scaled_df = pd.concat([df,vol_df],axis=1)
    return scaled_df

In [ ]:
def get_3D_surfacearea_and_sphericity(mask_img,scaled_df):
    surf_area_list = []
    sphericity_list = []
    vol = np.asarray(scaled_df['Volume (um^3)']).astype(np.float64)
    c = 0
    for i in np.asarray(scaled_df['label']).astype(int):
        obj = mask_img == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0,spacing=(voxel_size_z,voxel_size_x,voxel_size_y))
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        surf_area_list.append(surf_area)
        sphericity = (np.pi**(1/3)*((6*vol[c])**(2/3)))/surf_area
        sphericity_list.append(sphericity)
        c += 1
    surface_areas = pd.Series(surf_area_list,name='Surface_Area (um^2)')
    sphericities = pd.Series(sphericity_list,name='Sphericity')
    merged_df = pd.concat([scaled_df,surface_areas,sphericities], axis=1)
    return merged_df

Below cell contains the formula for calculating sphericity and the source of the code. Not required to run below cell for script to work.

In [ ]:
# def sphericity(mesh_volume, mesh_surface_area):
    
#     '''This definition of sphericity assumes that you are working in continuous space.
#     Parameters:
#     -----------
#     mesh_volume: integer or float value
#     mesh_surface_area: integer or float value
#     Returns:
#     --------
#     psi: a float value with range(0,1) reflecting the compactness of an object
#     Got this script from: https://github.com/BiAPoL/Bio-image_Analysis_with_Python/blob/main/05_feature_extraction/02_sphericity_and_solidity.ipynb
#     '''
#     numerator = (np.pi ** (1/3)) * ((6 * mesh_volume) ** (2/3))
#     denominator = mesh_surface_area
    
#     psi = numerator / denominator
    
#     return psi

In [ ]:
def save(save_path, img_name, norm_img, filtered_masks, merged_df):
    # make directories if they do not exist
    try:
        norm_path = os.mkdir(os.path.join(save_path,'norm_imgs'))
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        norm_path = os.path.join(save_path,'norm_imgs')
        dataframe_path = os.path.join(save_path,'measurements')
    imsave(os.path.join(masks_path,'filtered_masks_'+img_name[:-4]+'.tif'),filtered_masks,check_contrast=False)
    imsave(os.path.join(norm_path,'normalized_'+img_name[:-4]+'.tif'),norm_img)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

# Load in Data

## Two options for loading in data:
Note that options 1 and 2 expect the images to be in a tiff format. sk.io.imread cannot read in ims files

### Option 1: Specify the locations of all image containing folders:
Then merge into a long list of images and file names

In [ ]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [ ]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [ ]:
all_files = CMO_files+GMO_1ng_files+GMO_2ng_files # need this for getting image names later
all_imgs = CMO_imgs+GMO_1ng_imgs+GMO_2ng_imgs

### Option 2: Use a recursive search option to find all images in the parent directory:

In [ ]:
all_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/**/*.tif',recursive=True)) #finds all tiff images in the subdirectories of TIF_Files
all_imgs = list(map(sk.io.imread,all_files))

### Option 3: Read in Imaris files directly

In [ ]:
all_files = sorted(glob('./**/*.ims'))

In [ ]:
all_imgs = [ims(img,ResolutionLevelLock=0) for img in all_files]

In [ ]:
all_imgs = [img[0,0,:,:,:] for img in all_imgs]

In [ ]:
all_imgs[0].shape

# Run Pipeline

### Optional use of random integer generator if you want to test on single random images first

In [ ]:
import random
nums = range(len(all_img))
i = random.randint(min(nums),max(nums))

In [ ]:
tophat_radius = 10
img = normalize_images(all_img[i],tophat_radius) #change the tophat_radius to increase or decrease the background as needed

In [ ]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(img, diameter=70, channels=channels, z_axis=0,stitch_threshold=0.2,flow_threshold=0.5,cellprob_threshold=1.0)

In [ ]:
#View results of segmentation in Napari
viewer = napari.view_image(all_img[i], name='original img', scale = (4.55,0.3,0.3))
viewer.add_image(img,name='normalized img', scale = (4.55,0.3,0.3))
viewer.add_image(masks,name='masks', scale = (4.55,0.3,0.3))

### Run pipeline on all images

Set up a few parameters before running

Properties that can be measured with scikit-image can be found here: [Scikit-image Region Properties](https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops)

In [ ]:
tophat_radius = 10
voxel_size_x = 0.301
voxel_size_y = 0.301
voxel_size_z = 4.55
voxel = 0.301*0.301*4.55
save_path = 'E:/ParadaKusz_Lab/OIC-154_Microglia_Quantification/Segmentation_Outputs'
props = ('label','area','intensity_mean','intensity_min','intensity_max')

In [ ]:
norm_imgs = [normalize_images(img,tophat_radius) for img in tqdm(all_imgs)]

In [ ]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(norm_imgs, diameter=70, channels=channels, z_axis=0,stitch_threshold=0.2,flow_threshold=0.5,cellprob_threshold=1.0)

In [ ]:
for i in tqdm(range(len(norm_imgs))):
    img_name = os.path.basename(all_files[i])
    mask_img, df = get_measurements(masks[i],all_imgs[i],props)
    scaled_df = scaled_vol(df,voxel)
    merged_df = get_3D_surfacearea_and_sphericity(mask_img,scaled_df)
    save(save_path, img_name, norm_imgs[i], mask_img, merged_df)